In [ ]:
using Revise
using DrWatson
@quickactivate "SpikingNeuralNetworks"
using SpikingNeuralNetworks
SNN.@load_units
import SpikingNeuralNetworks: AdExParameter, IFParameter, IFConstParameter, AdExConstParameter
using Statistics, Random
using Plots

In [ ]:
## Neuron parameters
τm = 20ms
C = 300pF # Capacitance
R = τm / C

## Neuron parameters
τre = 1ms # Rise time for excitatory synapses
τde = 6ms # Decay time for excitatory synapses
τri = 0.5ms # Rise time for inhibitory synapses 
τdi = 2ms # Decay time for inhibitory synapses

# Input and synapse paramater
N = 1000
νe = 4.5Hz # Rate of external input to E neurons 
νi = 2.25Hz # Rate of external input to I neurons 
p_in = 0.2 #1.0 # 0.5 
σ_in_E = 1.78pF

σEE = 2.76pF # Initial E to E synaptic weight
σIE = 48.7pF # Initial I to E synaptic weight
σEI = 1.27pF # Synaptic weight from E to I
σII = 16.2pF # Synaptic weight from I to I

Random.seed!(23)
duration = 1000ms
pltdur = 100e1

In [ ]:
LKD_AdEx_exc = 
    AdExParameter(τm = 20ms, Vt = -52mV, Vr = -60mV, El = -70mV, R = 20ms/300pF, ΔT = 2mV, τw = 150ms, a = 4nS,
    b = 0.805pA, τabs = 1ms, τre = τre, τde = τde, τri = τri, τdi = τdi, E_i = -75mV, E_e = 0mV, At = 10mV, τT = 30ms)

E = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)

In [ ]:
LKD_IF_inh =
    IFParameter(τm = 20ms, Vt = -52mV, Vr = -60mV, El = -62mV, R = 20ms/300pF, ΔT = 2mV, 
    τre = τre, τde = τde, τri = τri, τdi = τdi, E_i = -75mV, E_e = 0mV, τabs = 1ms)

I = SNN.IF(; N = 1, param = LKD_IF_inh)

In [ ]:
Input_E = SNN.Poisson(; N = 600, param = SNN.PoissonParameter(; rate = νe)) # 
ProjE = SNN.SpikingSynapse(Input_E, E, :ge; σ = σ_in_E, p = p_in) # connection from input to E
Input_I = SNN.Poisson(; N = 150, param = SNN.PoissonParameter(; rate = νi))
ProjI = SNN.SpikingSynapse(Input_I, I, :ge; σ = σ_in_E, p = p_in)
iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()

EI = SNN.SpikingSynapse(E, I, :ge; σ = σEI, p = 1.0) 
IE = SNN.SpikingSynapse(I, E, :gi; σ = σIE, p = 1.0, param=iSTDPparams)
EE = SNN.SpikingSynapse(E, E, :ge; σ = σEE, p = 1.0, param=vSTDPparams)
II = SNN.SpikingSynapse(I, I, :gi; σ = σII, p = 1.0)

In [ ]:
P = [E, I, Input_E, Input_I]
C = [EE, II, EI, IE, ProjE, ProjI]

SNN.monitor([E, I], [:v, :fire]) 
SNN.train!(P, C; duration = duration)

In [ ]:
p1 = plot(SNN.vecplot([E], :v),
xlabel = "Time (ms)", 
ylabel = "Membrane Potential (mV)", 
title = "AdEx neuron coupled with IF neuron with Poisson inputs")
p2 = plot(SNN.vecplot([I], :v),
xlabel = "Time (ms)",
ylabel = "Membrane Potential (mV)", 
title = "IF neuron coupled with AdEx neuron with Poisson inputs")
plot(p1, p2, size=(1500, 450))

## Exploring weights

In [ ]:
frpre = 20Hz
frpost = 4.5Hz

Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
# Post = SNN.AdEx(; N = 1, param = LKD_AdEx_exc) 
Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

νe = 4.5Hz # Rate of external input to E neurons 
νi = 2.25Hz # Rate of external input to I neurons 

Input_Pre = SNN.Poisson(; N = 600, param = SNN.PoissonParameter(; rate = frpre)) # fr
ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to E
Input_Post = SNN.Poisson(; N = 150, param = SNN.PoissonParameter(; rate = frpost)) # fr
ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in)

iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()

PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σEI, p = 1.0) 
PostPost = SNN.SpikingSynapse(Post, Post, :gi; σ = σII, p = 1.0)
PrePre = SNN.SpikingSynapse(Pre, Pre, :ge; σ = σEE, p = 1.0, param=vSTDPparams)
PostPre = SNN.SpikingSynapse(Post, Pre, :gi; σ = σIE, p = 1.0, param=iSTDPparams)

P = [Pre, Post, Input_Pre, Input_Post]
C = [PostPre, PrePost, ProjPre, ProjPost, PrePre, PostPost]


SNN.monitor([Pre, Post], [:v, :fire]) 

W = zeros((2,1000)) # 700 seconds
for i in 1:1000
    SNN.train!(P, C; duration = 1second) # 
    W[1,i] = PostPre.W[1] # mean(PostPre.W)
    W[2,i] = PrePre.W[1] # mean(PrePre.W)
end

p1 = plot(W[2,:], label="Excitatory weight", ylims = (vSTDPparams.Wmin, vSTDPparams.Wmax))
p2 = plot(W[1,:], label="Inhibitory weight", ylims = (iSTDPparams.Wmin, iSTDPparams.Wmax))
plot(p1, p2,  size=(1000, 450), xlabel="Time (s)")

In [ ]:
# PrePost_weights = Vector{Float32}()
# PostPre_weights = Vector{Float32}()


# for fr in 1Hz:0.5Hz:50Hz

#     Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
#     # Post = SNN.IF(; N = 1, param = LKD_IF_inh) 
#     Post = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)

#     Input_Pre = SNN.Poisson(; N = 600, param = SNN.PoissonParameter(; rate = fr))
#     ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to E
#     Input_Post = SNN.Poisson(; N = 150, param = SNN.PoissonParameter(; rate = fr))
#     ProjPost = SNN.SpikingSynapse(Input_I, I, :ge; σ = σ_in_E, p = p_in)

#     PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σEI, p = 1.0, param=SNN.vSTDPParameter())
#     # IE = SNN.SpikingSynapse(I, E, :gi; σ = σIE, p = 1.0, param=SNN.iSTDPParameter())
#     # PrePre = SNN.SpikingSynapse(Pre, Pre, :ge; σ = σEE, p = 1.0, param=SNN.vSTDPParameter())
#     PostPre = SNN.SpikingSynapse(Post, Pre, :ge; σ = σIE, p = 1.0, param=SNN.vSTDPParameter()) # :gi

#     P = [Pre, Post, Input_Pre, Input_Post] # , Input_I
#     C = [PostPre, PrePost, ProjPre, ProjPost] #  IE, ProjI, PrePre, 

#     SNN.monitor([Pre, Post], [:v, :fire]) 
#     SNN.train!(P, C; duration = 5second)

#     push!(PrePost_weights, PrePost.W[1])
#     push!(PostPre_weights, PostPre.W[1])
# end

# p1 = plot(1:0.5:50, PrePost_weights, title="Weight pre-post", xlabel="Firing rate (Hz)", ylabel="Weight (pF)")
# p2 = plot(1:0.5:50, PostPre_weights, title="Weight post-pre", xlabel="Firing rate (Hz)", ylabel="Weight (pF)")
# plot(p1, p2, size=(1000, 450))